In [1]:
!pip install cornac

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 19.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from scipy.stats import entropy
import matplotlib.pyplot as plt
import random

import cornac
from cornac.eval_methods import RatioSplit
from cornac.data import TextModality
from cornac.data.text import BaseTokenizer
from cornac.data import Reader
from cornac.metrics import MAE,RMSE,MSE,FMeasure,Precision,Recall,NDCG,NCRR,MRR,AUC,MAP

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
docs_movielens, item_ids_movielens = cornac.datasets.movielens.load_plot()
feedback_movielens_100K = cornac.datasets.movielens.load_feedback(variant="100K",reader=Reader(item_set=item_ids_movielens))

movielens_item_text_modality = TextModality(
    corpus=docs_movielens,
    ids=item_ids_movielens,
    tokenizer=BaseTokenizer(sep="\t", stop_words="english"),
    max_vocab=8000,
    max_doc_freq=0.5,
)
# Define an evaluation method to split feedback into train and test sets
movieslens100K_ratio_split = RatioSplit(
    data=feedback_movielens_100K,
    test_size=0.2,
    exclude_unknowns=True,
    item_text=movielens_item_text_modality,
    verbose=True,
    seed=123,
    rating_threshold=1,
)

Data from https://static.preferred.ai/cornac/datasets/movielens/ml_plot.zip
will be cached into /root/.cornac/movielens/ml_plot.dat


0.00B [00:00, ?B/s]

Unzipping ...
File cached!
Data from http://files.grouplens.org/datasets/movielens/ml-100k/u.data
will be cached into /root/.cornac/ml-100k/u.data


0.00B [00:00, ?B/s]

File cached!
rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 943
Number of items = 1518
Number of ratings = 75846
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 941
Number of items = 1279
Number of ratings = 18925
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 943
Total items = 1518


### k=10

In [5]:
# Instantiate Conv_mf model
convmf_k10 = cornac.models.ConvMF(
    k=10,
    n_epochs=10,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=1,
    lambda_v=100,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[convmf_k10], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/10


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 29169157.33894 Elapsed: 5.8149s Converge: 2916915733893854432751313682154750584842200889217597308928.000000 
Epoch: 2/10


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 7671267.61777 Elapsed: 3.3892s Converge: 0.737008 
Epoch: 3/10


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 4285295.21207 Elapsed: 3.4192s Converge: 0.014150 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.7229 | 0.8561 | 0.9253 | 0.7000 | 0.0405 | 0.0354 | 0.0931 |  0.0370 |  0.0578 |       0.0315 |    0.0919 |   56.5238 |   3.3858



In [6]:
from cornac.utils import cache

# Download some information of MovieLens 100K dataset
user_df = pd.read_csv(
  cache("http://files.grouplens.org/datasets/movielens/ml-100k/u.user"),
  sep="|", names=["UserID", "Age", "Gender", "Occupation", "Zip Code"]
).set_index("UserID")

item_df = pd.read_csv(
  cache("http://files.grouplens.org/datasets/movielens/ml-100k/u.item"),
  sep="|", encoding="ISO-8859-1",
  names=["ItemID", "Title", "Release Date", "Video Release Date", "IMDb URL",
         "unknown", "Action", "Adventure", "Animation", "Children's", "Comedy",
         "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
         "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]
).set_index("ItemID").drop(columns=["Video Release Date", "IMDb URL", "unknown"])

Data from http://files.grouplens.org/datasets/movielens/ml-100k/u.user
will be cached into /root/.cornac/u.user


0.00B [00:00, ?B/s]

File cached!
Data from http://files.grouplens.org/datasets/movielens/ml-100k/u.item
will be cached into /root/.cornac/u.item


0.00B [00:00, ?B/s]

File cached!


In [8]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/master-project/conv_mf')


In [11]:
import diversity_movielens
def compute_avg_diversity(model):
    for i in range(len(set(movieslens100K_ratio_split.test_set.user_ids))):  # get the average result for all users
        result = []
        r = diversity_movielens.MovielensRetrieval(model=model, item_df=item_df, UIDX=i, TOPK=50)
        d = diversity_movielens.Diversity()
        recommendation = d.compute_dict_distr(r.get_recy())
        history = d.compute_dict_distr(r.get_history())
        result.append(d.compute_kl_divergence(recommendation, history, alpha=0.001))
    return np.mean(result)

In [12]:
data1 = {"model":"Conv_mf",
        "data":"movielens-100K",
        "k":10,
        "n_epochs":10,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":1,
        "lambda_v":100,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.7229,"MSE":0.8566,"RMSE":0.9255,"AUC":0.7143,"F1":0.0459,"MAP":0.0394,"MRR":0.1080,"NCRR":0.0446,"NDCG":0.0678,"Precision":0.0354,"Recall":0.1065,
        "category diversity":compute_avg_diversity(convmf_k10)}
print(data1['category diversity'])

0.1428636010152126


### k=50

In [13]:
# Instantiate Conv_mf model
convmf_k50 = cornac.models.ConvMF(
    k=50,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=1,
    lambda_v=100,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[convmf_k50], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 68997143.63355 Elapsed: 3.9787s Converge: 6899714363354588647458031375389016449433937555996245229568.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 12169900.12693 Elapsed: 4.8219s Converge: 0.823617 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 5352798.61360 Elapsed: 4.1532s Converge: 0.100176 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.7181 | 0.8314 | 0.9118 | 0.7286 | 0.0565 | 0.0458 | 0.1463 |  0.0610 |  0.0872 |       0.0432 |    0.1339 |   21.4897 |   3.0989



In [14]:
data2 = {"model":"Conv_mf",
        "data":"movielens-100K",
        "k":50,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":1,
        "lambda_v":100,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.7176,"MSE":0.8309,"RMSE":0.9115,"AUC":0.7270,"F1":0.0552,"MAP":0.0451,"MRR":0.1446,"NCRR":0.0598,"NDCG":0.0854,"Precision":0.0421,"Recall":0.1313,
        "category diversity":compute_avg_diversity(convmf_k50)}
print(data2['category diversity'])

0.1114737629826703


### k=100

In [15]:
# Instantiate Conv_mf model
convmf_k100 = cornac.models.ConvMF(
    k=100,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=1,
    lambda_v=100,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[convmf_k100], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 113729403.32339 Elapsed: 4.8688s Converge: 11372940332339380950187907006920983433370228458416037167104.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 15168876.76861 Elapsed: 5.8314s Converge: 0.866623 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 5783815.73820 Elapsed: 5.5602s Converge: 0.066444 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.7186 | 0.8310 | 0.9116 | 0.7118 | 0.0526 | 0.0458 | 0.1576 |  0.0638 |  0.0843 |       0.0404 |    0.1214 |   26.1475 |   2.5264



In [16]:
data3 = {"model":"Conv_mf",
        "data":"movielens-100K",
        "k":100,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":1,
        "lambda_v":100,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.7189,"MSE":0.8314,"RMSE":0.9118,"AUC":0.7059,"F1":0.0502,"MAP":0.0436,"MRR":0.1448,"NCRR":0.0585,"NDCG":0.0793,"Precision":0.0385,"Recall":0.1167,
        "category diversity":compute_avg_diversity(convmf_k100)}
print(data3['category diversity'])

0.08162838498246652


### k=200

In [17]:
# Instantiate Conv_mf model
convmf_k200 = cornac.models.ConvMF(
    k=200,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=1,
    lambda_v=100,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[convmf_k200], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 225678350.80393 Elapsed: 9.0690s Converge: 22567835080392835395163187635795464960019030135160008867840.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 18283563.03045 Elapsed: 6.8528s Converge: 0.918984 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 6021249.92121 Elapsed: 7.8124s Converge: 0.135074 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.7176 | 0.8265 | 0.9091 | 0.6924 | 0.0561 | 0.0452 | 0.1591 |  0.0650 |  0.0869 |       0.0433 |    0.1266 |   39.6734 |   2.5361



In [18]:
data4 = {"model":"Conv_mf",
        "data":"movielens-100K",
        "k":200,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":1,
        "lambda_v":100,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.7188,"MSE":0.8283,"RMSE":0.9101,"AUC":0.6899,"F1":0.0549,"MAP":0.0452,"MRR":0.1577,"NCRR":0.0647,"NDCG":0.0860,"Precision":0.0424,"Recall":0.1241,
        "category diversity":compute_avg_diversity(convmf_k200)}
print(data4['category diversity'])

0.11500040144678372


### k=300

In [19]:
# Instantiate Conv_mf model
convmf_k300 = cornac.models.ConvMF(
    k=300,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=1,
    lambda_v=100,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[convmf_k300], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 331181276.31393 Elapsed: 13.8709s Converge: 33118127631393219141709285641686794556520702934951416299520.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 21825550.68435 Elapsed: 13.8305s Converge: 0.934098 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 8486042.70202 Elapsed: 10.7115s Converge: 0.108558 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.7213 | 0.8314 | 0.9118 | 0.6973 | 0.0614 | 0.0490 | 0.1627 |  0.0693 |  0.0944 |       0.0473 |    0.1382 |   66.4690 |   2.6805



In [20]:
data5 = {"model":"Conv_mf",
        "data":"movielens-100K",
        "k":300,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":1,
        "lambda_v":100,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.7210,"MSE":0.8329,"RMSE":0.9126,"AUC":0.6811,"F1":0.0595,"MAP":0.0462,"MRR":0.1554,"NCRR":0.0657,"NDCG":0.0902,"Precision":0.0460,"Recall":0.1314,
        "category diversity":compute_avg_diversity(convmf_k300)}
print(data5['category diversity'])

0.09263514665584949


### k=400

In [21]:
# Instantiate Conv_mf model
convmf_k400 = cornac.models.ConvMF(
    k=400,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=1,
    lambda_v=100,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[convmf_k400], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 443814943.87600 Elapsed: 19.7763s Converge: 44381494387599509048845987902884273275093625541545735225344.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 25453274.70137 Elapsed: 20.3944s Converge: 0.942649 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 7185584.43623 Elapsed: 20.9294s Converge: 0.213752 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.7220 | 0.8398 | 0.9164 | 0.6485 | 0.0555 | 0.0434 | 0.1612 |  0.0659 |  0.0858 |       0.0429 |    0.1220 |   99.6932 |   2.6190



In [22]:
data6 = {"model":"Conv_mf",
        "data":"movielens-100K",
        "k":400,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":1,
        "lambda_v":100,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.7228,"MSE":0.8342,"RMSE":0.9134,"AUC":0.6791,"F1":0.0616,"MAP":0.0481,"MRR":0.1686,"NCRR":0.0703,"NDCG":0.0945,"Precision":0.0474,"Recall":0.1381,
        "category diversity":compute_avg_diversity(convmf_k400)}
print(data6['category diversity'])

0.09856911314472071


### k=300, lambda_u=10, lambda_v=100

In [23]:
# Instantiate Conv_mf model
convmf_u10_v100_k300 = cornac.models.ConvMF(
    k=300,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=10,
    lambda_v=100,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[convmf_u10_v100_k300], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 385902785.39687 Elapsed: 11.1044s Converge: 38590278539686820563094591611978791586796144768282477461504.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 24839080.50836 Elapsed: 10.4571s Converge: 0.935634 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 2338299.84703 Elapsed: 13.5553s Converge: 0.335493 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.7646 | 0.9222 | 0.9603 | 0.7305 | 0.0600 | 0.0481 | 0.1612 |  0.0677 |  0.0915 |       0.0467 |    0.1319 |   62.9102 |   2.5727



In [24]:
data7 = {"model":"Conv_mf",
        "data":"movielens-100K",
        "k":300,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":10,
        "lambda_v":100,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.7673,"MSE":0.9256,"RMSE":0.9621,"AUC":0.7381,"F1":0.0613,"MAP":0.0496,"MRR":0.1656,"NCRR":0.0698,"NDCG":0.0937,"Precision":0.0477,"Recall":0.1340,
        "category diversity":compute_avg_diversity(convmf_u10_v100_k300)}
print(data7['category diversity'])

0.08902311640355927


### k=300, lambda_u=100, lambda_v=100

In [25]:
# Instantiate Conv_mf model
convmf_u100_v100_k300 = cornac.models.ConvMF(
    k=300,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=100,
    lambda_v=100,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[convmf_u100_v100_k300], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 342575522.35103 Elapsed: 13.7723s Converge: 34257552235102626824100550184452617407660757714019419160576.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 21862282.27648 Elapsed: 13.6119s Converge: 0.936183 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 4278484.18236 Elapsed: 10.4647s Converge: 0.294276 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.9592 | 1.4280 | 1.1950 | 0.8088 | 0.0720 | 0.0642 | 0.1899 |  0.0869 |  0.1158 |       0.0552 |    0.1658 |   64.2467 |   3.8072



In [26]:
data8 = {"model":"Conv_mf",
        "data":"movielens-100K",
        "k":300,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":100,
        "lambda_v":100,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.9365,"MSE":1.3716,"RMSE":1.1712,"AUC":0.8036,"F1":0.0704,"MAP":0.0622,"MRR":0.1833,"NCRR":0.0832,"NDCG":0.1128,"Precision":0.0539,"Recall":0.1646,
        "category diversity":compute_avg_diversity(convmf_u100_v100_k300)}
print(data8['category diversity'])

0.08623593451790096


### k=300, lambda_u=1000, lambda_v=100

In [27]:
# Instantiate Conv_mf model
convmf_u1000_v100_k300 = cornac.models.ConvMF(
    k=300,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=1000,
    lambda_v=100,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[convmf_u1000_v100_k300], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 340222906.27076 Elapsed: 13.8238s Converge: 34022290627075800971118012527230703886895331310624651083776.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 24140193.34596 Elapsed: 13.9118s Converge: 0.929046 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 11742102.51530 Elapsed: 13.5549s Converge: 0.012887 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 1.7230 | 4.0579 | 2.0144 | 0.8094 | 0.0723 | 0.0632 | 0.2010 |  0.0887 |  0.1094 |       0.0577 |    0.1466 |   69.3526 |   2.4863



In [28]:
data9 = {"model":"Conv_mf",
        "data":"movielens-100K",
        "k":300,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":1000,
        "lambda_v":100,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":1.7119,"MSE":4.0162,"RMSE":2.0040,"AUC":0.8075,"F1":0.0709,"MAP":0.0632,"MRR":0.2032,"NCRR":0.0893,"NDCG":0.1085,"Precision":0.0566,"Recall":0.1430,
        "category diversity":compute_avg_diversity(convmf_u1000_v100_k300)}
print(data9['category diversity'])

0.04046200631333374


### k=300, lambda_u=1, lambda_v=10

In [29]:
# Instantiate Conv_mf model
convmf_u1_v10_k300 = cornac.models.ConvMF(
    k=300,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=1,
    lambda_v=10,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[convmf_u1_v10_k300], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 37026959.34077 Elapsed: 13.4085s Converge: 3702695934077234529627179948260644489283249609652551286784.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 7642227.55026 Elapsed: 10.8160s Converge: 0.793604 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 5331693.66568 Elapsed: 13.4682s Converge: 0.046551 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.7437 | 0.8855 | 0.9410 | 0.6737 | 0.0344 | 0.0303 | 0.0822 |  0.0288 |  0.0487 |       0.0261 |    0.0866 |   62.2934 |   2.8430



In [30]:
data10 = {"model":"Conv_mf",
        "data":"movielens-100K",
        "k":300,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":1,
        "lambda_v":10,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.7442,"MSE":0.8864,"RMSE":0.9415,"AUC":0.6665,"F1":0.0344,"MAP":0.0302,"MRR":0.0808,"NCRR":0.0287,"NDCG":0.0490,"Precision":0.0261,"Recall":0.0874,
        "category diversity":compute_avg_diversity(convmf_u1_v10_k300)}
print(data10['category diversity'])

0.09674799927711683


### k=300, lambda_u=1, lambda_v=1



In [31]:
# Instantiate Conv_mf model
convmf_u1_v1_k300 = cornac.models.ConvMF(
    k=300,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=1,
    lambda_v=1,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[convmf_u1_v1_k300], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 4284822.22353 Elapsed: 11.0423s Converge: 428482222353396845141892218267732562634831498040158715904.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 1770339.18436 Elapsed: 10.1582s Converge: 0.586835 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 1749941.89425 Elapsed: 13.6627s Converge: 0.002849 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.8556 | 1.1622 | 1.0780 | 0.5823 | 0.0191 | 0.0216 | 0.0522 |  0.0155 |  0.0251 |       0.0150 |    0.0431 |   62.1382 |   2.5912



In [32]:
data11 = {"model":"Conv_mf",
        "data":"movielens-100K",
        "k":300,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":1,
        "lambda_v":1,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.8557,"MSE":1.1613,"RMSE":1.0777,"AUC":0.5763,"F1":0.0188,"MAP":0.0213,"MRR":0.0528,"NCRR":0.0155,"NDCG":0.0247,"Precision":0.0148,"Recall":0.0423,
        "category diversity":compute_avg_diversity(convmf_u1_v1_k300)}
print(data11['category diversity'])

0.10896583789169488


### k=300, lambda_u=1, lambda_v=1000

In [33]:
# Instantiate Conv_mf model
convmf_u1_v1000_k300 = cornac.models.ConvMF(
    k=300,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=1,
    lambda_v=1000,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[convmf_u1_v1000_k300], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 3296369768.28186 Elapsed: 13.7605s Converge: 329636976828186017135686589360714863597703733881656087412736.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 154615486.61532 Elapsed: 13.6458s Converge: 0.953095 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 4205081.17965 Elapsed: 11.6087s Converge: 0.672121 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.7816 | 0.9585 | 0.9790 | 0.6467 | 0.0662 | 0.0533 | 0.2204 |  0.0908 |  0.1071 |       0.0519 |    0.1409 |   62.6070 |   4.3474



In [34]:
data12 = {"model":"Conv_mf",
        "data":"movielens-100K",
        "k":300,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":1,
        "lambda_v":1000,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.7848,"MSE":0.9628,"RMSE":0.9812,"AUC":0.6521,"F1":0.0672,"MAP":0.0536,"MRR":0.2206,"NCRR":0.0907,"NDCG":0.1077,"Precision":0.0526,"Recall":0.1432,
        "category diversity":compute_avg_diversity(convmf_u1_v1000_k300)}
print(data12['category diversity'])

0.09636677186979765


### k=300, lambda_u=100, lambda_v=10

In [35]:
# Instantiate Conv_mf model
convmf_u100_v10_k300 = cornac.models.ConvMF(
    k=300,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=100,
    lambda_v=10,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[convmf_u100_v10_k300], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 49172956.80316 Elapsed: 13.7753s Converge: 4917295680316267447042432385641612014106485821319277445120.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 8791826.12046 Elapsed: 13.5643s Converge: 0.821206 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 3464193.17294 Elapsed: 10.5472s Converge: 0.192503 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.7837 | 0.9824 | 0.9912 | 0.7408 | 0.0483 | 0.0370 | 0.0678 |  0.0315 |  0.0652 |       0.0366 |    0.1154 |   64.7993 |   3.4798



In [36]:
data13 = {"model":"Conv_mf",
        "data":"movielens-100K",
        "k":300,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":100,
        "lambda_v":10,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.7840,"MSE":0.9832,"RMSE":0.9916,"AUC":0.7419,"F1":0.0487,"MAP":0.0374,"MRR":0.0710,"NCRR":0.0324,"NDCG":0.0659,"Precision":0.0369,"Recall":0.1160,
        "category diversity":compute_avg_diversity(convmf_u100_v10_k300)}
print(data13['category diversity'])

0.12052259459972928


### k=300, lambda_u=10, lambda_v=100

In [37]:
# Instantiate Conv_mf model
convmf_u10_v100_k300 = cornac.models.ConvMF(
    k=300,
    n_epochs=5,
    cnn_epochs=5,
    cnn_bs=128,
    cnn_lr=0.001,
    lambda_u=10,
    lambda_v=100,
    emb_dim=200,
    max_len=300,
    filter_sizes=[3, 4, 5],
    num_filters=100,
    hidden_dim=200,
    dropout_rate=0.2,
    give_item_weight=True,
    trainable=True,
    verbose=True,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[convmf_u10_v100_k300], metrics=metrics, user_based=False
).run()


[ConvMF] Training started!
Epoch: 1/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 385902785.39687 Elapsed: 10.7077s Converge: 38590278539686820563094591611978791586796144768282477461504.000000 
Epoch: 2/5


Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 25441068.01237 Elapsed: 12.8001s Converge: 0.934074 
Epoch: 3/5


...

Optimizing CNN:   0%|          | 0/5 [00:00<?, ?it/s]

Loss: 2467106.70617 Elapsed: 13.4200s Converge: 0.277059 

[ConvMF] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
       |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
------ + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
ConvMF | 0.7647 | 0.9224 | 0.9604 | 0.7335 | 0.0598 | 0.0486 | 0.1636 |  0.0682 |  0.0916 |       0.0465 |    0.1314 |   65.1547 |   2.5992



In [38]:
data14 = {"model":"Conv_mf",
        "data":"movielens-100K",
        "k":300,
        "n_epochs":5,
        "cnn_epochs":5,
        "cnn_bs":128,
        "cnn_lr":0.001,
        "lambda_u":10,
        "lambda_v":100,
        "emb_dim":200,
        "max_len":300,
        "num_filters":100,
        "hidden_dim":200,
        "dropout_rate":0.2,
        "MAE":0.7673,"MSE":0.9256,"RMSE":0.9621,"AUC":0.7381,"F1":0.0613,"MAP":0.0496,"MRR":0.1656,"NCRR":0.0698,"NDCG":0.0937,"Precision":0.0477,"Recall":0.1340,
        "category diversity":compute_avg_diversity(convmf_u10_v100_k300)}
print(data14['category diversity'])

0.0893269468450629


In [39]:
data = [data1,data2,data3,data4,data5,data6,data7,data8,data9,data10,data11,data12,data13,data14]
df = pd.DataFrame(columns = data1.keys())
i=0
for d in data:
    df = pd.concat([df, pd.DataFrame(d,columns = list(d.keys()),index=[i])], ignore_index=False)
    i += 1

In [40]:
df

,model,data,k,n_epochs,cnn_epochs,cnn_bs,cnn_lr,lambda_u,lambda_v,emb_dim,...,RMSE,AUC,F1,MAP,MRR,NCRR,NDCG,Precision,Recall,category diversity
0,Conv_mf,movielens-100K,10,10,5,128,0.001,1,100,200,...,0.9255,0.7143,0.0459,0.0394,0.1080,0.0446,0.0678,0.0354,0.1065,0.142864
1,Conv_mf,movielens-100K,50,5,5,128,0.001,1,100,200,...,0.9115,0.7270,0.0552,0.0451,0.1446,0.0598,0.0854,0.0421,0.1313,0.111474
2,Conv_mf,movielens-100K,100,5,5,128,0.001,1,100,200,...,0.9118,0.7059,0.0502,0.0436,0.1448,0.0585,0.0793,0.0385,0.1167,0.081628
3,Conv_mf,movielens-100K,200,5,5,128,0.001,1,100,200,...,0.9101,0.6899,0.0549,0.0452,0.1577,0.0647,0.0860,0.0424,0.1241,0.115000
4,Conv_mf,movielens-100K,300,5,5,128,0.001,1,100,200,...,0.9126,0.6811,0.0595,0.0462,0.1554,0.0657,0.0902,0.0460,0.1314,0.092635
5,Conv_mf,movielens-100K,400,5,5,128,0.001,1,100,200,...,0.9134,0.6791,0.0616,0.0481,0.1686,0.0703,0.0945,0.0474,0.1381,0.098569
6,Conv_mf,movielens-100K,300,5,5,128,0.001,10,100,200,...,0.9621,0.7381,0.0613,0.0496,0.1656,0.0698,0.0937,0.0477,0.1340,0.089023
7,Conv_mf,movielens-100K,300,5,5,128,0.001,100,100,200,...,1.1712,0.8036,0.0704,0.0622,0.1833,0.0832,0.1128,0.0539,0.1646,0.086236
8,Conv_mf,movielens-100K,300,5,5,128,0.001,1000,100,200,...,2.0040,0.8075,0.0709,0.0632,0.2032,0.0893,0.1085,0.0566,0.1430,0.040462
9,Conv_mf,movielens-100K,300,5,5,128,0.001,1,10,200,...,0.9415,0.6665,0.0344,0.0302,0.0808,0.0287,0.0490,0.0261,0.0874,0.096748


In [41]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/master-project/conv_mf/convmf_movielens_100K.csv',header=-1,index=0)